In [3]:
import numpy as np

# -----------------------------
# Common Inputs
D = float(input("Distance (km): "))
tau = float(input("Traffic Index (0..1): "))   # 0 = no traffic, 1 = heavy traffic
n = int(input("Number of passengers: "))
C = float(input("Enter engine capacity (cc): ")) # Engine capacity (user input)
v_min = float(input("Enter minimum speed: "))
v_max = float(input("Enter maximum speed: "))

# -----------------------------
# Constants 
w_avg = 70.0 # avg weight (kg)
Pf = 102.0  # Fuel Price (currency per L)
Ct = 160.0  # delay constant
s = 0.05

# Preferred-speed / formula coefficients (unchanged)
c1 = 75.0
c2 = 5400.0
c3 = 4.0 / 70.0

# Small baseline fuel-speed coefficients (kept small)
ce1_base = 2e-4
ce2_base = 1e-5

# Engine-efficiency scaling for baseline coefficients
C_ref = 400.0
beta_eff = 0.20

# baseline K term
c0 = 0.005
c3_base = 0.001
c4 = 1e-5
c5 = 0.002

# Regularizer weight (same)
lambda_reg = 0.2

# Precompute scaling and baseline coefficients
B = Pf * D
eff_scale = 1.0 if C <= 0 else (C_ref / C) ** beta_eff
ce1 = ce1_base * eff_scale
ce2 = ce2_base * eff_scale
K = c0 + c3_base * tau + c4 * n * w_avg + c5 / (C if C > 0 else C_ref)


# -----------------------------
# Helper / model functions
def v_pref_formula(tau_val, C_val, n_val, w_avg_val):
    """Preferred speed based on traffic, engine capacity and passenger weight,
    clipped to [v_min, v_max]."""
    tau_clipped = np.clip(tau_val, 0.0, 1.0)
    C_safe = C_val if C_val > 0 else 1.0
    F = v_max - c1 * tau_clipped - (c2 / C_safe) - c3 * (n_val * w_avg_val)
    return float(np.clip(F, v_min, v_max))


def FC_baseline_per_km(v):
    """Baseline fuel consumption per km (quadratic in speed)."""
    return K + ce1 * v + ce2 * v**2


def f_base(v):
    """Baseline objective = fuel cost + time cost (without regularizer)."""
    if v <= 0:
        return float("inf")
    FC_v = FC_baseline_per_km(v)
    fuel_cost = B * FC_v
    time_cost = Ct * (D / v)
    const_delay = Ct * s * tau * D
    return fuel_cost + time_cost + const_delay


def f(v):
    """Full regularized objective f(v)."""
    vp = v_pref_formula(tau, C, n, w_avg)
    return f_base(v) + lambda_reg * (v - vp) ** 2


def fprime(v):
    """First derivative f'(v)."""
    base_deriv = B * (ce1 + 2.0 * ce2 * v) - Ct * D / (v**2)
    reg_deriv = 2.0 * lambda_reg * (v - v_pref_formula(tau, C, n, w_avg))
    return base_deriv + reg_deriv


def fdoubleprime(v):
    """Second derivative f''(v)."""
    base_hess = 2.0 * B * ce2 + 2.0 * Ct * D / (v**3)
    reg_hess = 2.0 * lambda_reg
    return base_hess + reg_hess


def proj(v):
    """Projection of v onto [v_min, v_max]."""
    return float(np.clip(v, v_min, v_max))


# -----------------------------
# Newton's Method
def newton_method(v0=40.0, tol=1e-6, max_iter=100, verbose=True):
    v = proj(v0)
    iterations = 0

    if verbose:
        print("\n=== Newton's Method ===")
        print(f"Computed v_pref (formula) = {v_pref_formula(tau, C, n, w_avg):.6f} km/h")
        print(f"Starting Newton from v0 (projected) = {v:.6f} km/h, lambda={lambda_reg}")
        print(f"Baseline fuel coefficients: K={K:.6e}, ce1={ce1:.6e}, ce2={ce2:.6e}")

    for k in range(1, max_iter + 1):
        g = fprime(v)
        H = fdoubleprime(v)

        if abs(g) < tol:
            iterations = k - 1
            break

        if abs(H) < 1e-14:
            if verbose:
                print("Hessian too small — stopping.")
            iterations = k - 1
            break

        step = g / H
        v_new = proj(v - step)

        if abs(v_new - v) < tol:
            v = v_new
            iterations = k
            if verbose:
                print(f"Converged by step size at iter {k}: |Δv| < {tol}")
            break

        v = v_new

    return v, iterations


# -----------------------------
# Steepest Descent
def steepest_descent(v0=40.0, tol=1e-6, max_iter=100, verbose=True):
    v = proj(v0)
    iterations = 0

    if verbose:
        print("\n=== Steepest Descent ===")
        print(f"Computed v_pref (formula) = {v_pref_formula(tau, C, n, w_avg):.6f} km/h")
        print(f"Starting Steepest Descent from v0 (projected) = {v:.6f} km/h, lambda={lambda_reg}")
        print(f"Baseline fuel coefficients: K={K:.6e}, ce1={ce1:.6e}, ce2={ce2:.6e}")

    for k in range(1, max_iter + 1):
        g = fprime(v)
        if abs(g) < tol:
            iterations = k - 1
            break

        # descent direction
        step_dir = -g
        alpha = 1.0
        f_v = f(v)

        # Armijo backtracking
        while True:
            v_trial = proj(v + alpha * step_dir)
            if f(v_trial) <= f_v + 1e-4 * alpha * g * step_dir:
                break
            alpha *= 0.5
            if alpha < 1e-12:
                break

        v_new = v_trial

        if abs(v_new - v) < tol:
            v = v_new
            iterations = k
            if verbose:
                print(f"Converged by step size at iter {k}: |Δv| < {tol}")
            break

        v = v_new

    return v, iterations


# -----------------------------
# BFGS Method (1D quasi-Newton)
def bfgs_method(v0=40.0, tol=1e-9, max_iter=200, verbose=True):
    x = np.array([proj(v0)], dtype=float)
    if verbose:
        print("\n=== BFGS (quasi-Newton) ===")
        print(f"Computed v_pref (formula) = {v_pref_formula(tau, C, n, w_avg):.6f} km/h")
        print(f"Starting BFGS from v0 (projected) = {x[0]:.6f} km/h, lambda={lambda_reg}")
        print(f"Baseline fuel coefficients: K={K:.6e}, ce1={ce1:.6e}, ce2={ce2:.6e}")

    # initialize gradient and inverse-H using analytic second derivative at v0
    g0 = float(fprime(x[0]))
    fpp0 = float(fdoubleprime(x[0]))

    inv_h0 = 1.0 / (abs(fpp0) + 1e-12)  # safe inverse Hessian scalar
    Hk = np.array([[inv_h0]], dtype=float)

    iter_count = 0

    # Armijo/backtracking parameters
    alpha0 = 1.0
    c1_ls = 1e-4
    rho = 0.5
    max_line_iters = 30

    for k in range(1, max_iter + 1):
        iter_count += 1
        g = np.array([fprime(x[0])], dtype=float)
        if abs(g[0]) < tol:
            break

        pk = -Hk.dot(g)  # direction (1x1)

        # ensure descent
        gTp = float(g.reshape(-1).dot(pk.reshape(-1)))
        if gTp >= 0:
            pk = -g
            gTp = float(g.reshape(-1).dot(pk.reshape(-1)))

        # line search on unprojected step, but evaluate f at projected candidate
        alpha = alpha0
        fx = f(x[0])
        found = False
        x_new = None

        for _ in range(max_line_iters):
            cand_unproj = x[0] + alpha * pk[0]
            cand_proj = proj(cand_unproj)
            f_cand = f(cand_proj)
            if f_cand <= fx + c1_ls * alpha * gTp:
                x_new = np.array([cand_proj], dtype=float)
                found = True
                break
            # if projection did nothing, reduce step
            if abs(cand_proj - x[0]) < 1e-12:
                alpha *= rho
                continue
            alpha *= rho

        if not found:
            alpha = 1e-6
            cand_proj = proj(x[0] + alpha * pk[0])
            x_new = np.array([cand_proj], dtype=float)

        s = x_new - x
        g_new = np.array([fprime(x_new[0])], dtype=float)
        y = g_new - g

        sty = float(s.reshape(-1).dot(y.reshape(-1)))
        if sty > 1e-12:
            rho_b = 1.0 / sty
            I = np.eye(1)
            term1 = I - rho_b * np.outer(s, y)
            term2 = I - rho_b * np.outer(y, s)
            Hk = term1.dot(Hk).dot(term2) + rho_b * np.outer(s, s)

        x = x_new

        if np.linalg.norm(s) < tol:
            break

    return float(x[0]), iter_count


# -----------------------------
# Trust-Region Method
def trust_region_optimization(v0, tol=1e-6, max_iter=100, verbose=True):
    v_k = proj(v0)
    iterations = 0

    # Trust-Region Parameters
    Delta_k = 10.0
    Delta_max = 50.0
    eta = 1e-4

    if verbose:
        vp = v_pref_formula(tau, C, n, w_avg)
        print("\n=== Trust-Region Optimization ===")
        print(f"Computed v_pref (formula) = {vp:.6f} km/h")
        print(f"Starting from v0 (projected) = {v_k:.6f} km/h, lambda={lambda_reg}")
        print(f"Baseline fuel coefficients: K={K:.6e}, ce1={ce1:.6e}, ce2={ce2:.6e}")
        print("-" * 50)

    for k in range(1, max_iter + 1):
        f_k = f(v_k)
        g_k = fprime(v_k)
        H_k = fdoubleprime(v_k)

        # 1. Stopping Criterion (Gradient Norm)
        if abs(g_k) < tol:
            iterations = k - 1
            break

        # Guard against zero Hessian
        if abs(H_k) < 1e-14:
            if verbose:
                print("Hessian too small — stopping.")
            iterations = k - 1
            break

        # 2. SOLVE SUBPROBLEM: Determine step p_k (Clamped Newton step)
        p_star = -g_k / H_k  # Newton step proposal
        p_k = np.clip(p_star, -Delta_k, Delta_k)  # Trust-region step (clamped to radius Delta_k)

        # 3. CALCULATE REDUCTIONS
        v_trial = v_k + p_k  # Trial point (before projection)
        p_actual = v_trial - v_k  # Actual step used for model

        ActRed = f_k - f(v_trial)  # Actual reduction
        PredRed = -(g_k * p_actual + 0.5 * H_k * p_actual ** 2)  # Predicted reduction

        # 4. REDUCTION RATIO (rho_k)
        if PredRed <= 0:
            rho_k = -1.0  # Indicates bad model or non-descent step
        else:
            rho_k = ActRed / PredRed

        # 5. UPDATE POSITION (v_k) AND RADIUS (Delta_k)
        v_next = v_k
        Delta_next = Delta_k

        step_accepted = False

        # Accept step if model is reliable enough and we get actual reduction
        if rho_k > eta and ActRed > 0:
            v_next = proj(v_trial)  # Project trial point onto [v_min, v_max]
            step_accepted = True

        # Radius Update
        if rho_k < 0.25:
            Delta_next = 0.25 * Delta_k  # Very poor agreement: shrink radius
        elif rho_k > 0.75 and np.abs(p_k) == Delta_k:
            Delta_next = min(2 * Delta_k, Delta_max)  # Very good agreement and step hit the boundary: expand radius

        # Stopping Criterion (Step Size)
        if step_accepted and abs(v_next - v_k) < tol:
            v_k = v_next
            iterations = k
            if verbose:
                print(f"Converged by step size at iter {k}: |Δv| < {tol}")
            break

        v_k = v_next
        Delta_k = Delta_next

    return v_k, iterations


# -----------------------------
# Common post-processing for all methods
def compute_costs(v_opt):
    """Return key cost metrics given optimal speed v_opt."""
    FC_base = FC_baseline_per_km(v_opt)            # L/km baseline
    baseline_fuel_used = FC_base * D               # total fuel used (L)

    # Target fuel logic (for D=10: tau=0 -> 0.37 L ; tau=1 -> 0.45 L)
    target_total_for_D10 = 0.37 + 0.08 * tau
    target_fuel_used = target_total_for_D10 * (D / 10.0)

    # Compute delta needed
    delta = target_fuel_used - baseline_fuel_used

    # Conditional forms:
    # if tau < 0.5: FC_v = FC_base - (1 - tau) * M  => M = -delta / ((1-tau)*D)
    # else:         FC_v = FC_base + tau * N        => N = delta / (tau*D)
    if tau < 0.5:
        denom = (1.0 - tau) * D
        M = 0.0
        if abs(denom) > 1e-12:
            M = -delta / denom
        FC_v = FC_base - (1.0 - tau) * M
    else:
        denom = tau * D
        N = 0.0
        if abs(denom) > 1e-12:
            N = delta / denom
        FC_v = FC_base + tau * N

    # Safety clamp
    FC_v = max(1e-12, FC_v)
    fuel_used = FC_v * D
    fuel_cost = Pf * fuel_used
    regularizer_penalty = lambda_reg * (v_opt - v_pref_formula(tau, C, n, w_avg)) ** 2
    total_cost = fuel_cost + regularizer_penalty

    return {
        "v_opt": v_opt,
        "fuel_used": fuel_used,
        "fuel_cost": fuel_cost,
        "regularizer_penalty": regularizer_penalty,
        "total_cost": total_cost,
    }


# -----------------------------
# Initial guess (user input; blank -> use v_pref)
v0_input = input("Initial Guess: ").strip()
if v0_input == "":
    v0 = v_pref_formula(tau, C, n, w_avg)
    print(f"Using v0 = v_pref_formula(...) = {v0:.6f} km/h as initial guess")
else:
    try:
        v0 = float(v0_input)
    except ValueError:
        print("Invalid initial guess; using v_pref_formula instead.")
        v0 = v_pref_formula(tau, C, n, w_avg)
v0 = proj(v0)

# -----------------------------
# Run all four methods
results = {}

v_newton, it_newton = newton_method(v0=v0, verbose=True)
results["Newton"] = (it_newton, compute_costs(v_newton))

v_sd, it_sd = steepest_descent(v0=v0, verbose=True)
results["Steepest Descent"] = (it_sd, compute_costs(v_sd))

v_bfgs, it_bfgs = bfgs_method(v0=v0, verbose=True)
results["BFGS"] = (it_bfgs, compute_costs(v_bfgs))

v_trust, it_trust = trust_region_optimization(v0=v0, verbose=True)
results["Trust Region"] = (it_trust, compute_costs(v_trust))

# -----------------------------
# Print individual method results
print("\n\n==== DETAILED RESULTS PER METHOD ====")
for name, (iters, metrics) in results.items():
    print(f"\n--- {name} ---")
    print(f"Iterations          : {iters}")
    print(f"Optimal Speed v*    : {metrics['v_opt']:.6f} km/h")
    print(f"Total Fuel used     : {metrics['fuel_used']:.6f} L")
    print(f"Fuel Cost           : {metrics['fuel_cost']:.2f}")
    print(f"Regularizer penalty : {metrics['regularizer_penalty']:.6f}")
    print(f"Total Fuel+Reg cost : {metrics['total_cost']:.2f}")

# -----------------------------
# Comparison summary
print("\n\n==== COMPARISON SUMMARY ====")
print(f"{'Method':<18} {'Iter':>6} {'v* (km/h)':>12} {'Fuel (L)':>12} {'Total Cost':>14}")
print("-" * 70)
for name, (iters, metrics) in results.items():
    print(f"{name:<18} {iters:>6d} {metrics['v_opt']:>12.4f} {metrics['fuel_used']:>12.4f} {metrics['total_cost']:>14.2f}")


Distance (km):  100
Traffic Index (0..1):  0.5
Number of passengers:  1
Enter engine capacity (cc):  400
Enter minimum speed:  25
Enter maximum speed:  100
Initial Guess:  34



=== Newton's Method ===
Computed v_pref (formula) = 45.000000 km/h
Starting Newton from v0 (projected) = 34.000000 km/h, lambda=0.2
Baseline fuel coefficients: K=6.205000e-03, ce1=2.000000e-04, ce2=1.000000e-05

=== Steepest Descent ===
Computed v_pref (formula) = 45.000000 km/h
Starting Steepest Descent from v0 (projected) = 34.000000 km/h, lambda=0.2
Baseline fuel coefficients: K=6.205000e-03, ce1=2.000000e-04, ce2=1.000000e-05

=== BFGS (quasi-Newton) ===
Computed v_pref (formula) = 45.000000 km/h
Starting BFGS from v0 (projected) = 34.000000 km/h, lambda=0.2
Baseline fuel coefficients: K=6.205000e-03, ce1=2.000000e-04, ce2=1.000000e-05

=== Trust-Region Optimization ===
Computed v_pref (formula) = 45.000000 km/h
Starting from v0 (projected) = 34.000000 km/h, lambda=0.2
Baseline fuel coefficients: K=6.205000e-03, ce1=2.000000e-04, ce2=1.000000e-05
--------------------------------------------------


==== DETAILED RESULTS PER METHOD ====

--- Newton ---
Iterations          : 4
Optim